# Scratchbook

In [22]:
from tika import parser
import pandas as pd
import glob

In [23]:
def parse_pdf(file):
    parsedPDF = parser.from_file(file)
    content = parsedPDF['content']
    contentlist = content.split('\n')
    contentlist = list(filter(lambda a: a != '', contentlist))
    contentlist = list(filter(lambda a: a != ' ', contentlist))
    return contentlist

In [24]:
def find_int(entry):
    try:
        out = isinstance(int(entry), int)
    except:
        out = False
    return out

Einzelne Positionen

In [25]:
def is_float(entry):
    try:
        out = isinstance(float(entry), float)
    except:
        out = False
    return out

In [26]:
def positions(t, contentlist, art_df):
    preis_list = []
    original_preis = []
    menge_list = []
    artikel_list = []
    gebinde_list = []

    for position in pd.Series(t[t].index):
        
        #article
        artikel_list.append(contentlist[position+1])
        
        #quantity
        menge_list.append(int(contentlist[position]))

        #binding
        txt = contentlist[position+2].split(' ')[-1]
        if not is_float(txt):
            text = contentlist[position+2]
        else:
            text = ' '.join(contentlist[position+2].split(' ')[:-1])
        gebinde_list.append(text)

        #price
        if position > pd.Series(t[t].index)[0]:
            if contentlist[position-1][:7] == 'http://':
                txt = contentlist[position-2]
            elif contentlist[position-1][:9] == 'Bestellnr':
                txt = contentlist[position-3]
            else:
                txt = contentlist[position-1]
            preis_list.append(float(txt.split(' ')[-1]))
            if not is_float(contentlist[position-2]):
                original_preis.append(float(txt.split(' ')[-1]))
            else:
                original_preis.append(float(contentlist[position-2]))

    #reduced price last article
    last_text = ''.join(contentlist[position:])
    last_end = last_text.find('Pfand')
    last_start = last_text[:last_end].rfind(' ')
    preis_list.append(float(last_text[last_start+1:last_end]))

    #original price last article
    for idx, entry in enumerate(contentlist):
        if entry.split(' ')[0] == 'Pfand':
            check = is_float(contentlist[idx-2])
            if not check:
                original_preis.append(float(last_text[last_start+1:last_end]))
            else:
                original_preis.append(float(contentlist[idx-2]))

    reduced_list = [i<j for i, j in zip(preis_list, original_preis)]

    art_df['position'] = range(1,len(pd.Series(t[t].index))+1)
    art_df['article'] = artikel_list
    einzelpreis_list = [round(i/j,2) for i, j in zip(preis_list, menge_list)]
    art_df['paid per unit'] = einzelpreis_list
    art_df['amount'] = menge_list
    art_df['paid per position'] = preis_list
    art_df['reduced'] = reduced_list

    einzelpreis_original_list = [i / j for i, j in zip(original_preis, menge_list)]
    art_df['original price per unit'] = einzelpreis_original_list
    art_df['original price per position'] = original_preis
    art_df['binding'] = gebinde_list

    unit_list = []
    for gebinde in gebinde_list:
        unit = [(i, c) for i,c in enumerate(gebinde) if c.isdigit()]
        if len(unit) > 0:
            unit_list.append(gebinde[[(i, c) for i,c in enumerate(gebinde) if c.isdigit()][-1][0]+1:].strip())
        else:
            unit_list.append('')
    measure_list = []
    for gebinde in gebinde_list:
        gebinde = gebinde.replace(',','.')
        txt = [(i, c) for i,c in enumerate(gebinde) if c.isdigit()]
        if len(txt) > 0:
            meas = gebinde[:[(i, c) for i,c in enumerate(gebinde) if c.isdigit()][-1][0]+1]
        else:
            meas = '1'
        mult =  meas.find(' x ')
        if mult > 0:
            meas = float(float(meas[:mult])*float(meas[mult+3:]))
        else:
            if is_float(meas):
                meas = float(meas)
            else:
                meas = float(meas.split(' ')[-1])
        measure_list.append(meas)

    art_df['quantity'] = measure_list
    art_df['unit'] = unit_list

    return art_df

Lieferdatum

In [27]:
def delivery(t, contentlist, art_df):
    end = ''.join(contentlist[0:pd.Series(t[t].index)[0]-2]).find('Klick')
    txt = ''.join(contentlist[0:pd.Series(t[t].index)[0]-2])[:end-2]
    start = txt.find('.')
    month = txt[start+1:]
    day = txt[:start][-2:].strip()
    art_df['delivery_day'] = day
    art_df['delivery_month'] = month
    return art_df

DeliveryID

In [28]:
def deliveryID(art_df, file):
    art_df['deliveryID'] = file
    return art_df

Bestellnummer

In [29]:
def order_nr(t, t2, contentlist, art_df):
    bestnr_list = []
    for bestnr in pd.Series(t2[t2].index):
        space = contentlist[bestnr].find(' ')
        bestnr_list.append(contentlist[bestnr][space+1:])
    
    idx_artikel = pd.Series(t[t].index)
    idx_bestellnr = pd.Series(t2[t2].index)
    
    bestell_list = []
    k = -1
    for i in range(len(idx_artikel)):
        if k < len(idx_bestellnr)-1 and i < len(idx_artikel)-1:
            if idx_artikel[i] > idx_bestellnr[k+1]:
                k += 1
        bestell_list.append(bestnr_list[k])
    art_df['order-nr'] = bestell_list

    return art_df, idx_bestellnr,bestnr_list
    

Hinzugefügt am

In [30]:
def added(idx_bestellnr, contentlist, bestnr_list, art_df):
    add_month_list = []
    add_day_list = []
    for idx in idx_bestellnr-1:
        dot = contentlist[idx].find('.')
        add_month_list.append(contentlist[idx][dot+2:])
        add_day_list.append(contentlist[idx][:dot][-2:].strip())
    add_df = pd.DataFrame({'order-nr.': bestnr_list, 'add_day': add_day_list, 'add_month': add_month_list})
    art_df = art_df.join(add_df.set_index('order-nr.'), on=('order-nr'))
    return art_df

In [31]:
def pfand(contentlist, art_df):
    pfand = 0.0
    for entry in contentlist:
        if entry[:19] == 'Eingereichtes Pfand':
            pfand = float(entry.split(' ')[-1])
    art_df['pfand'] = pfand
    return art_df

In [32]:

files = glob.glob('*.pdf')
table = pd.DataFrame(columns=['position','article','paid per unit','amount','paid per position','reduced','original price per unit','original price per position','binding','quantity','unit','delivery_day','delivery_month','deliveryID','order-nr','add_day','add_month','pfand'])

for file in files:
    art_df = pd.DataFrame()
    contentlist = parse_pdf(file)
    t = pd.DataFrame(contentlist).apply(lambda x: find_int(x), axis=1)
    art_df = positions(t,contentlist, art_df)
    art_df = delivery(t, contentlist, art_df)
    art_df = deliveryID(art_df, file)
    t2 = pd.DataFrame(contentlist).apply(lambda x: str(x)[5:14]=='Bestellnr', axis=1)
    art_df, idx_bestellnr, bestnr_list = order_nr(t, t2, contentlist, art_df)
    art_df = added(idx_bestellnr, contentlist, bestnr_list, art_df)
    art_df = pfand(contentlist, art_df)
    table = pd.concat([table, art_df], ignore_index=True)

table.to_excel('table.xlsx')

C:\Users\G23236\AppData\Local\Temp\ipykernel_3256\2114834104.py:3: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  out = isinstance(int(entry), int)
C:\Users\G23236\AppData\Local\Temp\ipykernel_3256\2114834104.py:3: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  out = isinstance(int(entry), int)


C:\Users\G23236\AppData\Local\Temp\ipykernel_3256\2114834104.py:3: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  out = isinstance(int(entry), int)


(Jahr)

Häufigste Artikel
(Preisverlauf)
Welche Produkte kaufen wir meistens zusammen?
Wie viel Geld geben wir pro Kategorie aus?

Kategorien: Getränke, Küchenartikel, Hygieneartikel, Tierprodukte, Milchprodukte, Fleisch und Fleischprodukte, Tiefkühlprodukte, Fertigprodukte und Suppen, Sonstiges, Frisches Obst, Getreide- und Backwaren, Gemüse, Süßwaren, Snacks und Knabbereien